In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0023026466369628906
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 24.57218792961797
200000 45.72099700721027
300000 58.82848988917017
400000 59.114783903966305
500000 59.55963218455498
600000 62.22402982442453
700000 58.92575705299931
800000 59.89705067166293
900000 57.69377771096955
1000000 59.72810147636369
1100000 41.13223787645532
1200000 34.49398235314991
1300000 59.08780846537291
1400000 56.91127491665761
1500000 58.713476764956795
1600000 57.74110625439165
1700000 61.75111768266699
1800000 58.858997294628026
1900000 57.55945920716852
2000000 60.029989524107414
2100000 58.32974283799465
2200000 45.88508817009488
2300000 31.03369071307784
2400000 56.72718974776286
2500000 59.17527730034854
2600000 55.8436441876871
2700000 57.53970262661517
2800000 60.59036443544667
2900000 55.67903325387064
3000000 56.36512185858

29700000 20.86613914352662
29800000 53.92029382531393
29900000 60.80060594089072
30000000 58.855892506066866
30100000 64.19301389161208
30200000 63.00361582105624
30300000 57.82161438399727
30400000 68.52833731777022
30500000 60.632205835645834
30600000 25.80147574094652
30700000 66.5595099489299
30800000 66.43824743045987
30900000 72.26524613477937
31000000 43.370536197830916
31100000 42.74536273225275
31200000 68.72724199306039
31300000 64.61016980797761
31400000 70.14982779241771
31500000 61.1829701633041
31600000 65.78939241947036
31700000 62.95265280161082
31800000 59.042583525665215
31900000 19.254916240918092
32000000 55.24910841050106
32100000 64.04789604904319
32200000 62.230876037322545
32300000 62.50084567164386
32400000 69.39930086502186
32500000 60.90216567057163
32600000 70.77074937133953
32700000 56.00761675491246
32800000 32.05312207594127
32900000 67.32553494747468
33000000 66.76739867141266
33100000 72.47302530308653
33200000 50.902287403312
33300000 35.79443712766363

59900000 46.907462567992816
60000000 57.715896360808294
60100000 54.56000396189165
60200000 51.12984651432393
60300000 55.13432968250397
60400000 42.98029456176258
60500000 23.53558585460349
60600000 52.23113687810618
60700000 51.568813431284056
60800000 52.45592579452943
60900000 51.54022219172552
61000000 57.58593069048651
61100000 26.89526879052563
61200000 55.27176684037458
61300000 62.241061116094535
61400000 55.87695903202771
61500000 61.989000085841056
61600000 57.92897530393608
61700000 61.36003222634251
61800000 61.267650257223075
61900000 58.93946360657123
62000000 40.90409495223477
62100000 40.47224220018056
62200000 58.04462840882285
62300000 52.85092018200221
62400000 51.20281306203497
62500000 54.91349057306269
62600000 44.06774734210685
62700000 26.384568581366555
62800000 52.18804388119995
62900000 52.5481243008956
63000000 52.32668948721154
63100000 53.10994831581825
63200000 57.67284514046838
63300000 24.120304566709287
63400000 58.24691478521631
63500000 60.939171836

90100000 51.96255378465138
90200000 52.9189808438578
90300000 52.20702930590267
90400000 54.433426237415475
90500000 52.2979211586991
90600000 49.25618707968334
90700000 47.77477995019073
90800000 26.309702038271414
90900000 40.50348692122986
91000000 48.65572879741949
91100000 46.20707389569368
91200000 26.65019748364407
91300000 40.64226311735696
91400000 47.83851570350302
91500000 47.429716261611254
91600000 30.121376799846757
91700000 39.100859314617225
91800000 47.044606537874344
91900000 49.990539655134405
92000000 50.45019504075361
92100000 53.75104933819147
92200000 52.448937294611895
92300000 52.095788323457576
92400000 57.528771909674575
92500000 55.38443059201221
92600000 57.69431785691055
92700000 53.09256134570295
92800000 50.586492095838366
92900000 51.41302320317236
93000000 31.925434388237516
93100000 38.56970907202159
93200000 48.413925273476394
93300000 50.251070861147326
93400000 24.201953416266257
93500000 40.542183814807835
93600000 51.642647192490045
93700000 49.3

119400000 47.80406624578745
119500000 50.252290352810455
119600000 17.902162902072035
119700000 47.56049044893683
119800000 12.800760927562894
119900000 45.046629410722694
120000000 23.43589051534977
120100000 51.376623628241404
120200000 46.38008554040631
120300000 52.67933560456059
120400000 47.46311927287102
120500000 51.55619236678156
120600000 51.37704795847973
120700000 51.29827342724341
120800000 59.39787600007445
120900000 54.50286369195998
121000000 59.233093807940605
121100000 52.33345549994663
121200000 48.38358117552587
121300000 53.86525706279228
121400000 48.36596486389423
121500000 53.782277343616364
121600000 50.29342098937927
121700000 47.391559691238584
121800000 22.46070499269407
121900000 45.46044320294091
122000000 13.989450231954374
122100000 38.11147848277583
122200000 30.669240160641642
122300000 49.449578678463446
122400000 47.98752919231957
122500000 54.761711100210285
122600000 48.38138095715407
122700000 51.8919644445767
122800000 51.15294210919342
122900000

...

638500000 2.062451059360703
638600000 2.108312747215722
638700000 1.1551189215578344
638800000 1.0965884031178321
638900000 0.7939640813943266
639000000 1.3065540320201323
639100000 2.229892322543246
639200000 1.6097226989259938
639300000 1.6513254568477997
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.0586359680510222
639500000 1.2151551949299493
639600000 1.7899305282116964
639700000 1.4254022729962956
639800000 1.7887463052608907
639900000 1.4723956311136006
640000000 1.6746110165786308
640100000 1.6246327821445703
640200000 1.642905561520642
640300000 1.7881216844115118
640400000 1.2258608046909114
640500000 1.4659638428992818
640600000 1.9003405971678045
640700000 2.1151159049855885
640800000 1.4211417518274765
640900000 1.7937571945081048
641000000 1.5910828968047717
641100000 1.6066018626589986
641200000 1.5877680969876686
641300000 1.6176205578248342
641400000 1.7630693212758348
641500000 1.4230294334429998
641600000 1.4233282850215794
641700

665300000 2.430145637123682
665400000 0.45888976054657693
665500000 0.3466387328292655
665600000 0.34684073099587165
665700000 0.42916432166060614
665800000 0.35528397983705734
665900000 0.3554224601429284
666000000 0.3523976221510721
666100000 0.3522541694986131
666200000 0.3525414751854578
666300000 0.3512070648952179
666400000 2.0088132107122463
666500000 3.1027831415098666
666600000 4.062920556340156
666700000 2.9833595222762432
666800000 3.2078521071475086
666900000 2.43996373658023
667000000 1.7633293888960095
667100000 1.8124633843126396
667200000 3.025467273225178
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 1.0536436972996375
667400000 0.29594008379473685
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 0.9498563467108919
667600000 1.5291336467413288
667700000 2.317639472300774
667800000 1.8281009578862661
667900000 2.9418944078784226
668000000 1.7011950408234002
668100000 0.43496907280710345
668200000 0.519016

691100000 1.264731096402204
691200000 0.5127928456799242
691300000 1.0990450095874937
691400000 1.2073288976958594
691500000 1.4669993522959481
691600000 1.5163358905408622
691700000 0.815872975491782
691800000 0.5715083537590762
691900000 1.0961135412626453
692000000 1.192000788932562
692100000 1.5129110934695322
692200000 2.02698338549325
692300000 0.9852098033112645
692400000 0.9524212331738353
692500000 1.130644976878367
692600000 1.3018802268857896
692700000 1.315385495781049
692800000 0.9709803407412456
692900000 0.7008037470639349
693000000 1.1247177211500976
693100000 1.241339861617699
693200000 1.247235159449198
693300000 0.9863712420126988
693400000 0.7461478330334504
693500000 0.923566866078331
693600000 1.6124958855517852
693700000 1.2124244232603802
693800000 1.1055820527371942
693900000 0.8101669577268732
694000000 0.65720289670332
694100000 1.706638139438971
694200000 1.258575151856626
694300000 1.2430238432622385
694400000 1.003464176185771
694500000 1.0574276690235593


718500000 0.26647122998824785
718600000 0.2861838076142748
best so far: 0
type: [1, 2, 5, 13, 1] 130
cases of this type: 109850
718700000 0.2760822574238499
best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.1088940781120499
718900000 0.12575362991313735
719000000 0.12487113854814172
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.2878231442277995
719200000 0.45493802628637825
719300000 0.4222223672589103
719400000 0.36040043911459446
719500000 0.36654948733695747
719600000 0.6074275323697528
719700000 0.3571460870726744
719800000 0.35611628455134103
719900000 0.32355241091310977
720000000 0.3643151709587998
720100000 0.23694413833551012
720200000 0.11608980897436937
720300000 0.11564691917100747
720400000 0.11546180637843807
720500000 0.2620693105159163
720600000 0.3450045253895912
7207000

738600000 0.20848441506925294
738700000 0.16239470764955083
738800000 0.18337261216995718
738900000 0.18139945691620774
739000000 0.1514653755734437
739100000 0.17175475813855134
739200000 0.17408398389981586
739300000 0.14778098502119316
739400000 0.17665664130532144
739500000 0.18049725338813002
739600000 0.21603299554024935
739700000 0.16957002262278198
739800000 0.16671720511002674
739900000 0.17592350533538262
740000000 0.17462395893662175
740100000 0.1529362159275724
740200000 0.18589569609263673
740300000 0.15530340545721055
740400000 0.1417473862713039
740500000 0.13646505564471484
740600000 0.16393791883387565
740700000 0.13905100927104289
740800000 0.13616960576798653
740900000 0.15033617685852646
741000000 0.17454965724996327
741100000 0.13350190600038966
741200000 0.1467103885984381
741300000 0.1473380350696749
741400000 0.15499585875896085
741500000 0.13843606717781423
741600000 0.1416077904244158
741700000 0.15865733923678532
741800000 0.13551819030446807
best so far: 0
t